# SageMaker Endpoint (Single Model Endpoint)
---

이제 **SageMaker 모델 호스팅 서비스인 SageMaker 엔드포인트**에 모델을 배포할 준비가 되었습니다. 

SageMaker 엔드포인트는 REST API를 통해 실시간 추론을 수행할 수 있는 완전 관리형 서비스입니다. 기본적으로 분산 컨테이너로 고가용성, 다중 모델 로딩, A/B 테스트를 위한 인프라 환경(EC2, 로드밸런서, 오토스케일링, 모델 아티팩트 로딩 등)이 사전 구축되어 있기에 몇 줄의 코드만으로 Endpoint가 자동으로 생성되기에, 모델을 프로덕션에 빠르게 배포할 수 있습니다.

SageMaker 빌트인 XGBoost를 사용하면 별도의 훈련/추론 스크립트 작성 없이 쉽게 모델을 훈련하고 엔드포인트로 배포할 수 있습니다. 하지만, 여러 가지 요인들로 인해 (예: SHAP 계산을 위한 피쳐 기여값 리턴, 추론값 및 추론 스코어 동시 리턴 등) 커스텀 추론 로직이 필요한 경우, SageMaker 빌트인 XGBoost 대신 SageMaker XGBoost 컨테이너를 사용할 수 있습니다.

이 노트북은 SageMaker XGBoost 컨테이너 상에서, 기본적인 추론 스크립트로 모델을 배포하는 법을 아래와 같은 목차로 진행합니다. 

완료 시간은 **20-30분** 정도 소요됩니다.

In [ ]:
import sagemaker

In [ ]:
%load_ext autoreload
%autoreload 2

## 1. parameter store 설정

In [ ]:
import boto3
import sagemaker
from utils.ssm import parameter_store

In [ ]:
region = boto3.Session().region_name
pm = parameter_store(region)

prefix = pm.get_params(key="PREFIX")
bucket_name = pm.get_params(key="-".join([prefix, "BUCKET-NAME"]))

role = sagemaker.get_execution_role()

### Upload model/source artifacts to S3
압축한 모델 아티팩트를 Amazon S3로 복사합니다.

In [ ]:
model_path = pm.get_params(key="-".join([prefix, "MODEL-PATH"]))

In [ ]:
!rm -rf model && mkdir -p model
!aws s3 cp {model_path} ./ 
!mv model.tar.gz ./model/
# !cd model && tar -xvf model.tar.gz

<br>

# 1. Create Model Serving Script

---

아래 코드 셀은 src 디렉토리에 SageMaker 추론 스크립트를 저장합니다.

#### Option 1.
- `model_fn(model_dir)`: S3의 `model_dir`에 저장된 모델 아티팩트를 로드합니다.
- `input_fn(request_body, content_type)`: 입력 데이터를 전처리합니다. `content_type`은 입력 데이터 종류에 따라 다양하게 처리 가능합니다. (예: `application/x-npy`, `application/json`, `application/csv`등)
- `predict_fn(input_object, model)`: `input_fn(...)`을 통해 들어온 데이터에 대해 추론을 수행합니다.
- `output_fn(prediction, accept_type)`: `predict_fn(...)`에서 받은 추론 결과를 후처리를 거쳐 프론트엔드로 전송합니다.

#### Option 2.
- `model_fn(model_dir)`: S3의 model_dir에 저장된 모델 아티팩트를 로드합니다.
- `transform_fn(model, request_body, content_type, accept_type)`: `input_fn(...), predict_fn(...), output_fn(...)`을 `transform_fn(...)`으로 통합할 수 있습니다.

In [ ]:
# %%writefile code/inference.py
# import os
# import json
# import time
# import torch
# import tarfile
# import argparse
# import numpy as np
# from io import BytesIO
# from datasets import load_from_disk
# from transformers import AutoModelForSequenceClassification, AutoTokenizer


# def model_fn(model_dir):
#     """
#     Deserialize and return fitted model.
#     """
#     tokenizer = AutoTokenizer.from_pretrained(model_dir)
#     model = AutoModelForSequenceClassification.from_pretrained(
#         model_dir,
#         num_labels=2
#     )
#     return (tokenizer, model)
                     

# def input_fn(request_body, request_content_type):
#     """
#     The SageMaker XGBoost model server receives the request data body and the content type,
#     and invokes the `input_fn`.
#     Return a DMatrix (an object that can be passed to predict_fn).
#     """
#     print("Content type: ", request_content_type)
#     if request_content_type == "application/x-npy":        
#         stream = BytesIO(request_body)
#         return stream.getvalue().decode()
#     elif request_content_type == "text/csv":
#         return request_body.rstrip("\n")
#     else:
#         raise ValueError(
#             "Content type {} is not supported.".format(request_content_type)
#         )
        

# def predict_fn(return_input_fn, return_model_fn):
#     """
#     SageMaker XGBoost model server invokes `predict_fn` on the return value of `input_fn`.

#     Return a two-dimensional NumPy array (predictions and scores)
#     """
#     start_time = time.time()
    
    
#     print(f"******************** return_input_fn : {return_input_fn}")
    
    
#     tokenizer, model = return_model_fn
#     encoded_input = tokenizer(return_input_fn, return_tensors='pt')
    
#     output = model(**encoded_input)
#     pred = torch.argmax(output.logits, dim=1)
    
#     print(f"******************** pred : {pred}")
    
#     print("--- Inference time: %s secs ---" % (time.time() - start_time))
#     return pred


# def output_fn(predictions, content_type="application/json"):
#     """
#     After invoking predict_fn, the model server invokes `output_fn`.
#     """
#     if content_type == "text/csv":
#         return predictions.tolist()[0]
#     elif content_type == "application/json":
#         outputs = json.dumps({'pred': predictions.tolist()[0]})        
        
#         return outputs
#     else:
#         raise ValueError("Content type {} is not supported.".format(content_type))

<br>

# 2. Deploy a trained model from Amazon S3
---

SageMaker API의 `Model` 클래스는 훈련한 모델을 서빙하기 위한 모델 아티팩트와 도커 이미지를 정의합니다. 
`Model` 클래스 인스턴스 호출 시 AWS에서 사전 빌드한 도커 이미지 URL을 직접 가져올 수도 있지만, Model의 자식 클래스로(예: `XGBoostModel`, `TensorFlowModel`) 초기화하면 파라메터에 버전만 지정하는 것만으로 편리하게 추론을 수행하는 환경을 정의할 수 있습니다.

<br>

## 2.1. Deploy to Local Environment: PytorchModel class

SageMaker 호스팅 엔드포인트로 배포하기 전에 로컬 모드 엔드포인트로 배포할 수 있습니다. 로컬 모드는 현재 개발 중인 환경에서 도커 컨테이너를 실행하여 SageMaker 프로세싱/훈련/추론 작업을 에뮬레이트할 수 있습니다. 추론 작업의 경우는 Amazon ECR의 딥러닝 프레임워크 기반 추론 컨테이너를 로컬로 가져오고(docker pull) 컨테이너를 실행하여(docker run) 모델 서버를 시작합니다.


```python
local_model_path = f'{os.getcwd()}/model'
ecr_uri = xgb_image_uri

# 도커 컨테이너 구동
!docker run --name xgb -itd -p 8080:8080 -v {local_model_path}:/opt/ml/model {ecr_uri} serve

# 실시간 호출 테스트 
!curl -X POST -H 'Content-Type: application/json' localhost:8080/invocations -d ...

# 도커 컨테이너 중지 및 삭제    
!docker stop xgb
!docker rm xgb
```

참고로 SageMaker SDK에서 `deploy(...)` 메소드로 엔드포인트 배포 시, 인스턴스 타입을 local 이나 local_gpu로 지정하면 위의 과정을 자동으로 수행할 수 있습니다.

```python
# 로컬 엔드포인트 배포
local_predictor = local_model.deploy(initial_instance_count=1, instance_type="local")

# 실시간 호출 테스트 
local_predictor.predict(...)

# 로컬 엔드포인트 삭제 (도커 컨테이너 중지 및 삭제)
local_predictor.delete_endpoint()
```

아래 코드를 보시면 아시겠지만, 지속적으로 업데이트되는 파이썬 버전&프레임워크 버전&트랜스포머 버전에 쉽게 대응할 수 있습니다. AWS에서 관리하고 있는 딥러닝 컨테이너(DLC) 목록을 아래 주소에서 확인해 보세요.
- https://github.com/aws/deep-learning-containers/blob/master/available_images.md

### Create Model

In [ ]:
instance_type='local'

In [ ]:
from pathlib import Path

# source_dir=f"file://{Path.cwd()}/src"

if instance_type in ['local', 'local_gpu']:
    from sagemaker.local import LocalSession
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    model_data=f"file://{Path.cwd()}/model/model.tar.gz"
else:
    sagemaker_session = sagemaker.session.Session()
    model_data=model_path

In [ ]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.model import Model
from sagemaker.image_uris import retrieve

pytorch_model = PyTorchModel(
    model_data=model_data,
    source_dir="code",
    role=role,
    entry_point="inference.py",
    framework_version='2.0.0',
    py_version='py310',
    sagemaker_session=sagemaker_session
)

### Create Endpoint

SageMaker SDK는 `deploy(...)` 메소드를 호출 시, `create-endpoint-config`와 `create-endpoint`를 같이 수행합니다. 좀 더 세분화된 파라메터 조정을 원하면 AWS CLI나 boto3 SDK client 활용을 권장 드립니다.

In [ ]:
pytorch_predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
)

### Check Docker

모델 서빙을 위한 도커 컨테이너가 구동되고 있음을 확인할 수 있습니다.

In [ ]:
!docker ps

### Prediction - boto3 SDK & application/x-npy

위의 코드 셀처럼 SageMaker SDK의 `predict(...)` 메소드로 추론을 수행할 수도 있지만, 이번에는 boto3의 `invoke_endpoint(...)` 메소드로 추론을 수행해 보겠습니다.
Boto3는 서비스 레벨의 저수준(low-level) SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 고수준(high-level) SDK인 SageMaker SDK와 달리 SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다.

[Note] `invoke_endpoint(...)` 호출을 위한 런타임 클라이언트 인스턴스 생성 시, 로컬 배포 모드에서는`sagemaker.local.LocalSagemakerRuntimeClient(...)`를 호출해야 합니다.

In [ ]:
payload = "This book offers useful, practical guidelines to support nonprofit managers in their efforts to maximize the effectiveness with which their organizations use their valuable resources.Nonprofit managers and leaders must advance their mission while balancing the agendas of trustees, funders, staff, and government. In this context, this group of expert authors explores core operating decisions that face all organizations and provides solutions that are unique to nonprofits of any size. Their chapters cover such key decisions as pricing of services, compensation of staff, outsourcing, fundraising expenditures, and investment and disbursement of funds."

In [ ]:
import json


if instance_type in ['local', 'local_gpu']:
    import sagemaker
    runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()
    endpoint_name = pytorch_model.endpoint_name
else:
    import boto3
    runtime_client = boto3.client('sagemaker-runtime')

In [ ]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/x-npy',
    Accept='application/json',
    Body=payload.encode()
)
print(f"******************* Prediction : {json.loads(response['Body'].read().decode())}")

### Prediction - boto3 SDK & text/csv

In [ ]:
import io
from io import StringIO
csv_file = io.StringIO()
payload = test_sentence
print(f"payload : {payload}")

In [ ]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='text/csv',
    Accept='application/json',
    Body=payload
)

print(f"******************* Prediction : {json.loads(response['Body'].read().decode())}")

### Local Mode Endpoint Clean-up

In [ ]:
pytorch_predictor.delete_endpoint()
pytorch_model.delete_model()

<br>

## 2.2. Deploy to Hosting Instance

로컬 모드에서 충분히 디버깅했으면 실제 호스팅 인스턴스로 배포할 차례입니다. 코드는 거의 동일하며, `instance_type`만 다르다는 점을 주목해 주세요! 

In [ ]:
instance_type='ml.m5.xlarge'

In [ ]:
from pathlib import Path

# source_dir=f"file://{Path.cwd()}/src"

if instance_type in ['local', 'local_gpu']:
    from sagemaker.local import LocalSession
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    model_data=f"file://{Path.cwd()}/model/model.tar.gz"
else:
    sagemaker_session = sagemaker.session.Session()
    model_data=model_path
model_data

### Create Model

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(
    model_data=model_data,
    source_dir="code",
    role=role,
    entry_point="inference.py",
    framework_version='2.0.0',
    py_version='py310',
    sagemaker_session=sagemaker_session
)

### Create Endpoint

SageMaker SDK는 `deploy(...)` 메소드를 호출 시, `create-endpoint-config`와 `create-endpoint`를 같이 수행합니다. 좀 더 세분화된 파라메터 조정을 원하면 AWS CLI나 boto3 SDK client 활용을 권장 드립니다.

In [ ]:
from time import strftime

model_name = 'distilbert-base-uncased'

create_date = strftime("%m%d-%H%M%s")
endpoint_name=f'finetune-{model_name}-{create_date}'

pytorch_predictor = pytorch_model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type=instance_type, 
    wait=False
)

### Wait for the endpoint jobs to complete

엔드포인트가 생성될 때까지 기다립니다. 엔드포인트가 가리키는 호스팅 리소스를 프로비저닝하는 데에 몇 분의 시간이 소요됩니다. 

In [ ]:
from IPython.core.display import display, HTML
def make_endpoint_link(region, endpoint_name, endpoint_task):
    endpoint_link = f'<b><a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">{endpoint_task} Review Endpoint</a></b>'   
    return endpoint_link 
        
endpoint_link = make_endpoint_link(region, pytorch_predictor.endpoint_name, '[Deploy model from S3]')
display(HTML(endpoint_link))

In [ ]:
sagemaker_session.wait_for_endpoint(pytorch_predictor.endpoint_name, poll=5)

### Prediction - SageMaker SDK & text/csv
샘플 데이터에 대해 추론을 수행합니다.

In [ ]:
from sagemaker.serializers import CSVSerializer, NumpySerializer
from sagemaker.deserializers import JSONDeserializer
pytorch_predictor.serializer = CSVSerializer()
pytorch_predictor.deserializer = JSONDeserializer() 

outputs = pytorch_predictor.predict(test_sentence)
outputs

### Prediction - boto3 SDK & application/x-npy

위의 코드 셀처럼 SageMaker SDK의 `predict(...)` 메소드로 추론을 수행할 수도 있지만, 이번에는 boto3의 `invoke_endpoint(...)` 메소드로 추론을 수행해 보겠습니다.
Boto3는 서비스 레벨의 저수준(low-level) SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 고수준(high-level) SDK인 SageMaker SDK와 달리 SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다.

In [ ]:
runtime_client = boto3.client('sagemaker-runtime')
endpoint_name = pytorch_model.endpoint_name
payload = test_sentence.encode()

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/x-npy',
    Accept='application/json',
    Body=payload
)

print(json.loads(response['Body'].read().decode()))

### Prediction - boto3 SDK & text/csv

In [ ]:
payload = test_sentence

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='text/csv',
    Accept='application/json',
    Body=payload
)

print(json.loads(response['Body'].read().decode()))

### (Optional) Endpoint Clean-up

In [ ]:
# pytorch_predictor.delete_endpoint()
# pytorch_model.delete_model()